In [ ]:
rm(list=ls()) # clear workspace
suppressPackageStartupMessages(require(argparse))
suppressPackageStartupMessages(require(tidyverse))
suppressPackageStartupMessages(require(DropletUtils))

In [2]:
run.emptyDrops <- function(m, lower, FDR.thresh=0.01, ...){
  # wrapper for emptyDrops function
  #   m = counts dataframe in genes x cells format
  #   lower = number of total UMI counts below which all barcodes are considered empty droplets
  #   FDR.thresh = threshold above which barcodes are labeled as empty droplets (default 0.01)
  #   ... = additional args to pass to emptyDrops() function
  start.time <- Sys.time()
  
  result <- emptyDrops(m = m, lower = lower, ...)
  
  out <- data.frame(result@listData) %>%
    mutate(empty = as.numeric(FDR > FDR.thresh)) %>%
    mutate(empty = replace_na(empty, 1)) %>%
    mutate(barcode = result@rownames)
  
  print(Sys.time() - start.time)
  return(out)
}

In [3]:
read.counts <- function(path, transpose=F, row.names=NULL){
  start.time <- Sys.time()
  if(str_detect(string = path, pattern = '.csv')){
    counts <- read.csv(path, row.names=row.names)
  }else if(str_detect(string = path, pattern = '.tsv')){
    counts <- read.csv(path, sep = '\t', row.names=row.names)
  }
  
  if(transpose){
    counts <- t(counts)
  }
  print(Sys.time() - start.time) # print file reading time
  return(counts)
}

In [4]:
#library(Seurat)
library(hdf5r)


Attaching package: ‘hdf5r’

The following object is masked from ‘package:SummarizedExperiment’:

    values

The following object is masked from ‘package:GenomicRanges’:

    values

The following object is masked from ‘package:IRanges’:

    values

The following object is masked from ‘package:S4Vectors’:

    values

The following object is masked from ‘package:purrr’:

    flatten_df



In [5]:
#sampleS3 <- read.counts('data/2771-AS-3-ACAGTG_S3.h5ad', transpose=T)

In [6]:
#sampleS3 <- ReadH5AD('data/2771-AS-3-ACAGTG_S3.h5ad')

In [7]:
#library(tidyverse)

In [8]:
#read.table("NonScanpyVersion//Example_dataset.csv.gz", row.names = 1, sep = ",", header = T) -> d

In [9]:
#output <- run.emptyDrops(sampleS3, lower = 500)

In [10]:
data_in <- h5file("data/2771-AS-3-ACAGTG_S3.h5ad")
data_in_raw <- data_in[['X']][,] #raw count data, make sure this is raw counts/ints

In [11]:
data_in_obs = data_in[['obs']][] #load obs
data_in_var = data_in[['var']][] #load vars
rownames(data_in_raw) = data_in_var$index #set labels
colnames(data_in_raw) = data_in_obs$index #set labels

In [12]:
#data_in_seurat = CreateSeuratObject(data_in_raw,project = 'seurat_obj_name') #create seurat object
#data_in_seurat$batch = data_in_obs$batch #set further labels from h5ad annotations if necessary

In [13]:
#data_in_seurat@assays$RNA@counts

In [14]:
dim(data_in_raw)

[1] 38459 14189

In [15]:
emptyDrops_thresh_output <- run.emptyDrops(data_in_raw, lower = 400)

Time difference of 35.81419 secs


In [16]:
emptyDrops_thresh_output

Total,LogProb,PValue,Limited,FDR,empty,barcode
<int>,<dbl>,<dbl>,<lgl>,<dbl>,<dbl>,<chr>
9881,-9434.998,9.999e-05,TRUE,0.000000000,0,bcEFAX
9825,-10575.418,9.999e-05,TRUE,0.000000000,0,bcFOEC
7338,-7746.110,9.999e-05,TRUE,0.000000000,0,bcIJXQ
10237,-16400.995,9.999e-05,TRUE,0.000000000,0,bcFQMC
10901,-18395.173,9.999e-05,TRUE,0.000000000,0,bcHHFW
10441,-15387.323,9.999e-05,TRUE,0.000000000,0,bcHMGZ
11439,-17611.167,9.999e-05,TRUE,0.000000000,0,bcGJGH
8706,-18505.348,9.999e-05,TRUE,0.000000000,0,bcGVOF
8392,-11779.140,9.999e-05,TRUE,0.000000000,0,bcGYED


In [17]:
sum(emptyDrops_thresh_output$empty) # total number of empty drops based on emptyDrops threshold

[1] 11097

In [18]:
write.csv(emptyDrops_thresh_output,"testing/emptyDrops-thresh-2771-S3.csv", row.names = FALSE)